In [2]:
import json
from datasets import Dataset

In [3]:
print(tokenized.keys())

NameError: name 'tokenized' is not defined

In [4]:
import transformers
print(transformers.__version__)

4.51.3


In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
dataset = load_dataset('json', data_files='merged_nutrition_dataset.json')

# Choose the model (Flan-T5 or Gemma)
model_name = "google/flan-t5-base"  # or "google/gemma-2b" for Gemma
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to format the input for instruction tuning
def format_input(example):
    prompt = f"Instruction: {example['instruction']}\nInput: {example['input']}\nOutput: {example['output']}"
    return {"text": prompt}

# Apply formatting
dataset = dataset.map(format_input)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512  # Adjust based on your model's max length
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["instruction", "input", "output", "text"])  # Clean up
tokenized_dataset.set_format("torch")

# Split into train and validation sets
train_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)["train"]
eval_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)["test"]

In [6]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSeq2SeqLM  # or AutoModelForCausalLM for Gemma

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # Use AutoModelForCausalLM for Gemma

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor
    target_modules=["q", "v"],  # Target attention layers (adjust for Gemma if needed)
    lora_dropout=0.1,  # Dropout for regularization
    bias="none",
    task_type="SEQ_2_SEQ_LM"  # Use "CAUSAL_LM" for Gemma
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify the number of trainable parameters

trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


In [8]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    num_train_epochs=3,  # Adjust based on dataset size
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    fp16=False, 
    gradient_accumulation_steps=4 # Enable if using GPU
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

C:\Users\padha\AppData\Local\Temp\ipykernel_89228\3436812911.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


: 

In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# 1. Load and preprocess the dataset
dataset = load_dataset('json', data_files='merged_nutrition_dataset.json')

# Choose Flan-T5-base
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Format input for instruction tuning
def format_input(example):
    # Input for encoder: instruction + input
    input_text = f"Instruction: {example['instruction']}\nInput: {example['input']}"
    # Target for decoder: output
    target_text = example['output']
    return {"input_text": input_text, "target_text": target_text}

dataset = dataset.map(format_input)

# Tokenize the dataset
def tokenize_function(example):
    # Tokenize input (for encoder)
    model_inputs = tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # Tokenize target (for decoder)
    model_targets = tokenizer(
        example["target_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # Return input_ids, attention_mask, and labels
    model_inputs["labels"] = model_targets["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["instruction", "input", "output", "input_text", "target_text"])
tokenized_dataset.set_format("torch")

# Split into train and validation sets
train_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)["train"]
eval_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)["test"]

# 2. Set up the model with LoRA
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 3. Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Reduced for CPU
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    fp16=False,  # CPU
    gradient_accumulation_steps=4  # Helps with low memory
)

# 4. Initialize and train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()

# 5. Save the model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

Map:   0%|          | 0/1533 [00:00<?, ? examples/s]

Map:   0%|          | 0/1533 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# 1. Load and preprocess the dataset
dataset = load_dataset('json', data_files='merged_nutrition_dataset.json')

# Choose Flan-T5-base
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Format input for instruction tuning
def format_input(example):
    # Ensure instruction and input are strings, handle None or invalid types
    instruction = str(example['instruction']) if example['instruction'] is not None else ""
    input_text = str(example['input']) if example['input'] is not None else ""
    target_text = str(example['output']) if example['output'] is not None else ""
    # Input for encoder: instruction + input
    formatted_input = f"Instruction: {instruction}\nInput: {input_text}"
    return {"input_text": formatted_input, "target_text": target_text}

dataset = dataset.map(format_input)

# Verify dataset integrity
def check_dataset(example):
    # Skip examples with empty or non-string input_text/target_text
    return (isinstance(example['input_text'], str) and
            isinstance(example['target_text'], str) and
            example['input_text'].strip() != "" and
            example['target_text'].strip() != "")

dataset = dataset.filter(check_dataset)

# Tokenize the dataset
def tokenize_function(examples):
    # Ensure batched inputs are lists of strings
    input_texts = [text if isinstance(text, str) else "" for text in examples["input_text"]]
    target_texts = [text if isinstance(text, str) else "" for text in examples["target_text"]]
    
    # Tokenize input (for encoder)
    model_inputs = tokenizer(
        input_texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # Tokenize target (for decoder)
    model_targets = tokenizer(
        target_texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # Add labels for seq2seq training
    model_inputs["labels"] = model_targets["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["instruction", "input", "output", "input_text", "target_text"])
tokenized_dataset.set_format("torch")

# Split into train and validation sets
train_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)["train"]
eval_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)["test"]

# 2. Set up the model with LoRA
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 3. Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Reduced for CPU
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    fp16=False,  # CPU
    gradient_accumulation_steps=4  # Helps with low memory
)

# 4. Initialize and train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()

# 5. Save the model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

C:\Users\padha\AppData\Local\Temp\ipykernel_54720\2227751085.py:102: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


c:\Users\padha\Documents\edunet\project\Ai Nutrition assistant\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


: 